In [46]:
# needed imports
from flask import Flask, render_template, request, jsonify
from sqlmodel import SQLModel, Field, create_engine, Session
from typing import Optional
import uuid
import hashlib
from module import check_strategy_timeframe, list_strategy_folder, download_data, run_backtest, read_latest_backtest
from models import TradeData, BacktestData
from config import * 
# extra imports
from pprint import pprint
import pandas as pd
import sqlite3


In [47]:
unique_id = uuid.uuid4().hex[:16]
data_hash = "test_hash"

In [48]:
# Read latest backtest
backtest_dict = read_latest_backtest(results_dir=LOCAL_RESULTS_DIR, backtest_uuid=unique_id, backtest_hash=data_hash)
# pprint(backtest_data)

# Filter out fields with unsupported types
backtest_data = {k: v for k, v in backtest_dict.items() if not isinstance(v, (list, dict))}

# Filter out fields with unsupported types
skipped_fields = {k: v for k, v in backtest_dict.items() if isinstance(v, (list, dict))}

if skipped_fields:
    print("Skipped fields:", skipped_fields)


Skipped fields: {'pairlist': ['ATOM/USDT'], 'minimal_roi': {'0': 0.01}}


In [49]:
# Create the database engine
engine = create_engine("sqlite:///backtest_data_sqlmodel.db")

# Create the database tables
SQLModel.metadata.create_all(engine)


# Create a BacktestData instance
backtest_instance = BacktestData(**backtest_data)

# Insert the data into the database
with Session(engine) as session:
    session.add(backtest_instance)
    session.commit()